In [1]:
#Install Dependencies
!pip install streamlit==0.79.0
!pip install --upgrade streamlit
!pip install -r requirements.txt
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.3 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=61e12413b096b1bc0e493819d72ca88bb7a6552e977076bc1280a851f391c903
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 36.9 MB/s 

In [2]:
%%writefile app.py

import streamlit as st
import os
import tensorflow_hub as hub
from utils import load_img, transform_img, tensor_to_image, imshow
import tensorflow as tf
import numpy as np
from PIL import Image

PAGE_CONFIG = {"page_title":"ML Artist", "layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

# Only use the below code if you have low resources.
os.environ['CUDA_VISIBLE_DEVICES'] = ""
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

# For supressing warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

st.write("""
# Machine Learned Artist
""")

@st.cache
def load_model():
    hub_model = hub.load(
        'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
    return hub_model


model_load_state = st.text('Loading Model...')
model = load_model()
# Notify the reader that the data was successfully loaded.
model_load_state.text('Loading Model...  Done!')


content_image, style_image = st.columns(2)

with content_image:
    st.write('## Content Image...')
    chosen_content = st.radio(
        '  ',
        ("Upload", "URL"))
    if chosen_content == 'Upload':
        # st.write(f"You choosed {chosen_content}!")
        content_image_file = st.file_uploader(
            "Pick a Content Image", type=("png", "jpg"))
        try:
            content_image_file = content_image_file.read()
            content_image_file = transform_img(content_image_file)
        except:
            pass
    elif chosen_content == 'URL':
        # st.write(f"You choosed {chosen_content}!")
        url = st.text_input('URL for Content Image')
        try:
            content_path = tf.keras.utils.get_file(
                os.path.join(os.getcwd(), 'content.jpg'), url)
        except:
            pass
        try:
            content_image_file = load_img(content_path)

        except:
            pass
    try:
        st.write('Selected Content Image...')
        st.image(imshow(content_image_file))
    except:
        pass

with style_image:
    st.write('## Style Image...')
    chosen_style = st.radio(
        ' ',
        ("Upload", "URL"))
    if chosen_style == 'Upload':
        # st.write(f"You choosed {chosen_style}!")
        style_image_file = st.file_uploader(
            "Pick a Style Image", type=("png", "jpg"))
        try:
            style_image_file = style_image_file.read()
            style_image_file = transform_img(style_image_file)
        except:
            pass
    elif chosen_style == 'URL':
        # st.write(f"You choosed {chosen_style}!")
        url = st.text_input('URL for Style Image')
        try:
            style_path = tf.keras.utils.get_file(
                os.path.join(os.getcwd(), 'style.jpg'), url)
        except:
            pass
        try:
            style_image_file = load_img(style_path)

        except:
            pass
    try:
        st.write('Selected Style Image...')
        st.image(imshow(style_image_file))
    except:
        pass

predict = st.button('Start Neural Art-Style Transfer...')


if predict:
    try:
        stylized_image = model(tf.constant(
            content_image_file), tf.constant(style_image_file))[0]

        final_image = tensor_to_image(stylized_image)
    except:
        stylized_image = model(tf.constant(
            tf.convert_to_tensor(content_image_file[:, :, :, :3])
        ),
            tf.constant(
                tf.convert_to_tensor(style_image_file[:, :, :, :3])
        )
        )[0]

        final_image = tensor_to_image(stylized_image)

    st.write('Resultant Image...')
    st.image(final_image)

    try:
        # Delete style.jpg and content.jpg
        os.remove("style.jpg")
        os.remove("content.jpg")
    except:
        pass

hide_streamlit_style = """
  
  """

st.markdown(hide_streamlit_style, unsafe_allow_html=True) 

st.write('Made by Team GOOSE')

Writing app.py


In [3]:
%%writefile utils.py
import tensorflow as tf
import numpy as np
from PIL import Image


def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img


def transform_img(img):
    max_dim = 512
    img = tf.image.decode_image(img)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img


def imshow(image, title=None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    image = np.squeeze(image)
    return image


def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return Image.fromarray(tensor)

Writing utils.py


In [4]:
!ngrok authtoken 21IbdIzRR0vD4f0Y5kwg9VSjKvm_21tAbwZRhMDKj3NdpzeTu

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
from pyngrok import ngrok

In [6]:
# !nohub streamlit run app.py
!streamlit run app.py&>/dev/null&

In [7]:
!pgrep streamlit

488


In [8]:
public_url = ngrok.connect(port='8501')

In [9]:
public_url

'http://3496-34-80-194-203.ngrok.io'